In [1]:
"""
The design of this comes from here:
http://outlace.com/Reinforcement-Learning-Part-3/
"""

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import RMSprop
from keras.layers.recurrent import LSTM
from keras.callbacks import Callback

# Adding this per a suggestion by Tim Kelch.
# https://medium.com/@trkelch/this-post-is-great-possibly-the-best-tutorial-explanation-ive-found-thus-far-cf78886b5378#.w473ywtbw


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))


def neural_net(num_sensors, params, load=''):
    model = Sequential()

    # First layer.
    model.add(Dense(
        params[0], kernel_initializer='lecun_uniform', input_shape=(num_sensors,)
    ))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    # Second layer.
    model.add(Dense(params[1], kernel_initializer='lecun_uniform'))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    # Output layer.
    model.add(Dense(3, kernel_initializer='lecun_uniform'))
    model.add(Activation('linear'))

    rms = RMSprop()
    model.compile(loss='mse', optimizer=rms)

    if load:
        model.load_weights(load)

    return model


def lstm_net(num_sensors, load=False):
    model = Sequential()
    model.add(LSTM(
        output_dim=512, input_dim=num_sensors, return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(output_dim=512, input_dim=512, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim=3, input_dim=512))
    model.add(Activation("linear"))
    model.compile(loss="mean_squared_error", optimizer="rmsprop")

    return model

Using Theano backend.


In [2]:
from carGame import carGame
import numpy as np
import random
import csv
from neuralNetwork import neural_net, LossHistory
import os.path
import timeit

NUM_INPUT = 2
GAMMA = 0.9  # Forgetting.
TUNING = False  # If False, just use arbitrary, pre-selected params.


def train_net(model, params):

    filename = params_to_filename(params)

    observe = 1000  # Number of frames to observe before training.
    epsilon = 1
    train_frames = 10000  # Number of frames to play.
    batchSize = params['batchSize']
    buffer = params['buffer']

    # Just stuff used below.
    max_car_distance = 0
    car_distance = 0
    t = 0
    data_collect = []
    replay = []  # stores tuples of (S, A, R, S').

    loss_log = []

    # Create a new game instance.
    game_state = carGame()

    # Get initial state by doing nothing and getting the state.
    _, state = game_state.update(0)

    # Let's time it.
    start_time = timeit.default_timer()

    # Run the frames.
    while t < train_frames:

        t += 1
        car_distance += 1

        # Choose an action.
        if random.random() < epsilon or t < observe:
            action = np.random.randint(0, 3)  # random
        else:
            # Get Q values for each action.
            qval = model.predict(state, batch_size=1)
            action = (np.argmax(qval))  # best

        # Take action, observe new state and get our treat.
        reward, new_state = game_state.update(action)

        # Experience replay storage.
        replay.append((state, action, reward, new_state))

        # If we're done observing, start training.
        if t > observe:

            # If we've stored enough in our buffer, pop the oldest.
            if len(replay) > buffer:
                replay.pop(0)

            # Randomly sample our experience replay memory
            minibatch = random.sample(replay, batchSize)

            # Get training values.
            X_train, y_train = process_minibatch(minibatch, model)

            # Train the model on this batch.
            history = LossHistory()
            model.fit(
                X_train, y_train, batch_size=batchSize,
                nb_epoch=1, verbose=0, callbacks=[history]
            )
            loss_log.append(history.losses)

        # Update the starting state with S'.
        state = new_state

        # Decrement epsilon over time.
        if epsilon > 0.1 and t > observe:
            epsilon -= (1/train_frames)

        # We died, so update stuff.
        if reward == -1000:
            print('crash reported');
            # Log the car's distance at this T.
            data_collect.append([t, car_distance])

            # Update max.
            if car_distance > max_car_distance:
                max_car_distance = car_distance

            # Time it.
            tot_time = timeit.default_timer() - start_time
            fps = car_distance / tot_time

            # Output some stuff so we can watch.
            print("Max: %d at %d\tepsilon %f\t(%d)\t%f fps" %
                  (max_car_distance, t, epsilon, car_distance, fps))

            # Reset.
            car_distance = 0
            start_time = timeit.default_timer()

        # Save the model every 25,000 frames.
        if t % 250 == 0:
            model.save_weights('saved-models/' + filename + '-' +
                               str(t) + '.h5',
                               overwrite=True)
            print("Saving model %s - %d" % (filename, t))

    # Log results after we're done all frames.
    log_results(filename, data_collect, loss_log)


def log_results(filename, data_collect, loss_log):
    # Save the results to a file so we can graph it later.
    with open('results/sonar-frames/learn_data-' + filename + '.csv', 'w') as data_dump:
        wr = csv.writer(data_dump)
        wr.writerows(data_collect)

    with open('results/sonar-frames/loss_data-' + filename + '.csv', 'w') as lf:
        wr = csv.writer(lf)
        for loss_item in loss_log:
            wr.writerow(loss_item)


def process_minibatch(minibatch, model):
    """This does the heavy lifting, aka, the training. It's super jacked."""
    X_train = []
    y_train = []
    # Loop through our batch and create arrays for X and y
    # so that we can fit our model at every step.
    for memory in minibatch:
        # Get stored values.
        old_state_m, action_m, reward_m, new_state_m = memory
        # Get prediction on old state.
        old_qval = model.predict(old_state_m, batch_size=1)
        # Get prediction on new state.
        newQ = model.predict(new_state_m, batch_size=1)
        # Get our best move. I think?
        maxQ = np.max(newQ)
        y = np.zeros((1, 3))
        y[:] = old_qval[:]
        # Check for terminal state.
        if reward_m != -500:  # non-terminal state
            update = (reward_m + (GAMMA * maxQ))
        else:  # terminal state
            update = reward_m
        # Update the value for the action we took.
        y[0][action_m] = update
        X_train.append(old_state_m.reshape(NUM_INPUT,))
        y_train.append(y.reshape(3,))

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train


def params_to_filename(params):
    return str(params['nn'][0]) + '-' + str(params['nn'][1]) + '-' + \
            str(params['batchSize']) + '-' + str(params['buffer'])


def launch_learn(params):
    filename = params_to_filename(params)
    print("Trying %s" % filename)
    # Make sure we haven't run this one.
    if not os.path.isfile('results/sonar-frames/loss_data-' + filename + '.csv'):
        # Create file so we don't double test when we run multiple
        # instances of the script at the same time.
        open('results/sonar-frames/loss_data-' + filename + '.csv', 'a').close()
        print("Starting test.")
        # Train.
        model = neural_net(NUM_INPUT, params['nn'])
        train_net(model, params)
    else:
        print("Already tested.")


if __name__ == "__main__":
    if TUNING:
        param_list = []
        nn_params = [[164, 150], [256, 256],
                     [512, 512], [1000, 1000]]
        batchSizes = [40, 100, 400]
        buffers = [10000, 50000]

        for nn_param in nn_params:
            for batchSize in batchSizes:
                for buffer in buffers:
                    params = {
                        "batchSize": batchSize,
                        "buffer": buffer,
                        "nn": nn_param
                    }
                    param_list.append(params)

        for param_set in param_list:
            launch_learn(param_set)

    else:
        nn_param = [164, 150]
        params = {
            "batchSize": 100,
            "buffer": 50000,
            "nn": nn_param
        }
        model = neural_net(NUM_INPUT, nn_param)
        train_net(model, params)

/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(164, input_shape=(2,), kernel_initializer="lecun_uniform")`
  params[0], init='lecun_uniform', input_shape=(num_sensors,)
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_initializer="lecun_uniform")`
  model.add(Dense(params[1], init='lecun_uniform'))
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kernel_initializer="lecun_uniform")`
  model.add(Dense(3, init='lecun_uniform'))


(+) Game initialized successfully
-48533.64461735476
-48274.99361637668
-48020.05130786072
-47768.87705009109
-47770.59325568993
-47769.77320875707
-47482.83202067685
-47200.597822006785
-46920.2165251394
-46641.72153308661
-46365.14682356244
-46090.526952306485
-45817.8970559083
-45547.29285409402
-45278.75065143401
-45012.30733842969
-44748.000391935195
-44485.86787486875
-44225.948435166414
-43968.28130392983
-43970.05358226474
-43621.094150627425
-43277.41254499697
-43280.046215633374
-43279.20916273614
-43278.360478485636
-42841.54800862367
-42410.48430977328
-42413.98983710088
-41979.98215231683
-41983.440303461764
-41982.57111526186
-41981.66823888159
-41496.023116364806
-41499.997559133866
-41499.08135058761
-41498.16492505561
-40959.789845892556
-40426.802448844304
-39893.81891981984
-39360.83941675787
-38827.86410630641
-38294.89316443077
-37761.9267770733
-37766.36547072536
-37267.48361717702
-37271.52645545881
-36772.94287480789
-36279.65687313353
-35786.69191650981
-35790.

-36713.64665695706
-36869.09982247006
-37028.40170016296
-37025.88909407843
-37109.25739992068
-37198.262260192816
-37196.521984442705
-37273.02303674143
-37271.41666663048
-37352.00556243084
-37435.848523600216
-37523.68461313856
-37615.485837503285
-37711.223219104795
-37810.86683854693
-37914.38587747783
-37912.499666470096
-37911.63860783407
-37923.9416441929
-37940.5538295666
-37939.52573215982
-37865.221319352495
-37796.75409247729
-37796.58207269038
-37795.7189381546
-37731.6337775764
-37673.33722428961
-37620.1619381103
-37619.807217853064
-37442.43051604168
-37443.31823741597
-37269.88800564242
-37270.706637973504
-37101.27734399705
-36937.59676323292
-36938.3594525046
-36646.04920342458
-36648.050430333285
-36647.12944771472
-500
crash reported
Max: 174 at 440	epsilon 1.000000	(113)	18.471528 fps
-38917.459073040074
-38610.480487686116
-38306.15922042749
-38308.290322415705
-37968.3274606666
-37633.19543133735
-37299.51934751638
-36967.338631956976
-36969.74939168073
-36968.8

-34187.43818282909
-34563.728963835565
-34559.003983219074
-34558.104295560384
-35046.286763160686
-35040.482364399024
-35039.559068920935
-35038.61041244413
-35532.24709870006
-36022.22027632951
-36513.40193632334
-37005.743993023774
-37499.20077346514
-37993.728874857785
-38489.28703151007
-38985.835990517524
-39483.33839559676
-39981.758678487015
-40481.0629573847
-40981.21894191524
-40975.22902462404
-41481.258325225135
-41475.28418451073
-41474.34802460386
-41977.503766710615
-42475.78730857745
-42974.222941322725
-42968.2540394937
-43467.86771328811
-43962.49385261554
-44457.1282953768
-44951.77076805056
-45008.792755944894
-45007.9790573453
-45507.62461179185
-45501.79362268264
-45500.980624694144
-45997.05938049856
-45991.28468512707
-46484.54948674543
-46973.865767434574
-47464.126348752434
-47955.30225444188
-48447.36563095154
-48441.55988213562
-48440.72480971457
-48439.872427304195
-48837.04234525664
-49232.09611640772
-49227.27103442778
-49628.10858071883
-49623.2478833229

/Applications/anaconda/lib/python3.6/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


...

-51782.16610665396
-51819.58217044508
-51860.373067314984
-51904.53083540096
-51952.04688435889
-52002.91200358909
-52057.116371003525
-52114.649562315106
-500
crash reported
Max: 425 at 8786	epsilon 0.221400	(12)	7.025497 fps
-51007.50281736418
-51006.4840462943
-51005.59517167872
-51004.72083371752
-51003.8193911281
-50881.70294456846
-50764.41617700827
-50650.72769727962
-50650.97468242779
-50539.76097969433
-50539.957717911304
-50356.128431074474
-50357.051896345925
-50356.138193808314
-50355.21099628265
-50354.2840955261
-50032.28640630479
-50034.687444545576
-50033.8694973913
-50033.03806037317
-49623.42995406772
-49626.69481895489
-49625.864228177794
-49625.01980818571
-49624.17554551409
-49138.05491792349
-49142.04365384134
-49141.18578228482
-48641.74369039965
-48645.85172224782
-48644.97943783068
-48644.07796255559
-48131.520772165
-47624.65649814852
-47628.874892104956
-47117.4696077291
-46611.783080983776
-46106.71462621314
-45602.28476991992
-45098.514929966135
-44595.4274

-49665.55221293281
-49659.775157245116
-50155.973594596966
-50647.47941988504
-50641.69589754629
-51135.59852187905
-500
crash reported
Max: 425 at 9115	epsilon 0.188500	(9)	7.694985 fps
-48622.16909748006
-49105.55046865604
-49589.95212431434
-50075.34444050582
-50561.69887201922
-51048.98790653139
-51537.185020910605
-500
crash reported
Max: 425 at 9123	epsilon 0.187700	(8)	7.635783 fps
-500
crash reported
Max: 425 at 9124	epsilon 0.187600	(1)	7.575525 fps
-500
crash reported
Max: 425 at 9125	epsilon 0.187500	(1)	7.591121 fps
-500
crash reported
Max: 425 at 9126	epsilon 0.187400	(1)	7.451742 fps
-500
crash reported
Max: 425 at 9127	epsilon 0.187300	(1)	7.640032 fps
-50988.29885145667
-51472.68220995659
-500
crash reported
Max: 425 at 9130	epsilon 0.187000	(3)	7.526493 fps
-500
crash reported
Max: 425 at 9131	epsilon 0.186900	(1)	7.636864 fps
-500
crash reported
Max: 425 at 9132	epsilon 0.186800	(1)	7.486824 fps
-500
crash reported
Max: 425 at 9133	epsilon 0.186700	(1)	7.787700 fps
-5

-52454.598273751435
-52766.09356885299
-500
crash reported
Max: 425 at 9429	epsilon 0.157100	(10)	7.605918 fps
-50822.40934597104
-51122.37781629985
-51423.64611577958
-51726.191529880896
-52029.99177730759
-52335.02500286253
-52641.2697703152
-52948.70505528047
-53257.310238117425
-500
crash reported
Max: 425 at 9439	epsilon 0.156100	(10)	7.638671 fps
-51344.4499853472
-51641.38591213955
-51939.64361052954
-51935.79898630833
-52173.20586357322
-52410.38482395461
-52649.66015102242
-52891.0033895127
-53134.386357578696
-53379.78115145172
-500
crash reported
Max: 425 at 9450	epsilon 0.155000	(11)	7.821087 fps
-50763.84152932061
-50979.31570831104
-51197.13763837742
-51417.27747743653
-51639.70557756389
-51864.392494037165
-52091.30899384746
-52320.42606369084
-52551.71491745267
-52785.14700319764
-53020.69400967873
-53258.327872379006
-53498.02077909937
-53739.74517510674
-500
crash reported
Max: 425 at 9465	epsilon 0.153500	(15)	7.615441 fps
-51391.69616615439
-51605.68825663072
-51822

-34563.00893111471
-34478.94054729749
-34400.361700663
-34327.310048023326
-34259.820905964305
-34197.92716869291
-34141.659230445344
-34141.33005017705
-500
crash reported
Max: 425 at 9806	epsilon 0.119400	(10)	6.037405 fps
-34188.91661843961
-34074.046521666976
-33964.555976240634
-33860.49711435616
-33761.920107316015
-33668.873054048985
-33581.40187196677
-33499.55019068832
-33423.35924916514
-33352.8677967388
-33288.11199865512
-33229.125346548375
-33175.938574392545
-33128.579580396356
-500
crash reported
Max: 425 at 9821	epsilon 0.117900	(15)	7.326571 fps
-32956.106509056146
-32873.89046852621
-32797.44130995791
-32726.799399938518
-32662.002373201623
-32603.085035913762
-32550.079274962693
-32503.01397378139
-500
crash reported
Max: 425 at 9830	epsilon 0.117000	(9)	7.575478 fps
-32330.84360954229
-32248.249159906005
-32171.53177999781
-32100.733556224102
-32035.893683913273
-31977.048362667407
-31924.230698314313
-31877.470612059453
-31836.794757404212
-500
crash reported
Max: 

FileNotFoundError: [Errno 2] No such file or directory: 'results/sonar-frames/learn_data-164-150-100-50000.csv'

In [ ]:
"""
Once a model is learned, use this to play it.
"""

from carGame import carGame
import numpy as np
from neuralNetwork import neural_net

NUM_SENSORS = 2


def play(model):

    car_distance = 0
    game_state = carGame()

    # Do nothing to get initial.
    _, state = game_state.update(0)

    # Move.
    while True:
        car_distance += 1

        # Choose action.
        action = (np.argmax(model.predict(state, batch_size=1)))

        # Take action.
        _, state = game_state.update(action)

        # Tell us something.
        if car_distance % 1000 == 0:
            print("Current distance: %d frames." % car_distance)


if __name__ == "__main__":
    saved_model = 'saved-models/164-150-100-50000-10000.h5'
    model = neural_net(NUM_SENSORS, [164, 150], saved_model)
    play(model)

/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(164, input_shape=(2,), kernel_initializer="lecun_uniform")`
  params[0], init='lecun_uniform', input_shape=(num_sensors,)
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_initializer="lecun_uniform")`
  model.add(Dense(params[1], init='lecun_uniform'))
/Users/abhinav/Documents/NeuralNetworksMotionPlanning/neuralNetwork.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kernel_initializer="lecun_uniform")`
  model.add(Dense(3, init='lecun_uniform'))


(+) Game initialized successfully
4.846030199428071
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
